In [58]:
import matplotlib.pyplot as plt
import os

import pandas
import numpy as np
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.bn2graph as bnPlot
import pyAgrum.lib.dynamicBN as gdyn

## Creating a network using PyAgrum

### Create the empty BN network

In [39]:
bn=gum.BayesNet('Diabetes Remission')
G = bn.add(gum.LabelizedVariable('Glycemia','superior to 6.5 ?',2))
I = bn.add(gum.LabelizedVariable('Insulin','Insulin taken ?',2))
O = bn.add(gum.LabelizedVariable('Other_drugs','Other drugs taken ?',2))
R = bn.add(gum.LabelizedVariable('Remission','Remission or not ?',2))
for link in [(G, I),(G, O),(I, R), (O, R)]:
    bn.addArc(*link)
print(bn)

BN{nodes: 4, arcs: 4, domainSize: 16, dim: 18}


### Set the probability table

In this TME 0 is yes and 1 is no

In [40]:
bn.cpt(G).fillWith([0.5,0.5])
bn.cpt(I)[:]=[ [0.9,0.1],[0.1,0.9]]
bn.cpt(O)[:]=[ [0.75,0.25],[0.3,0.7]]
bn.cpt(R)[{'Insulin': 0, 'Other_drugs': 0}] = [0.1, 0.9]
bn.cpt(R)[{'Insulin': 1, 'Other_drugs': 0}] = [0.6, 0.4]
bn.cpt(R)[{'Insulin': 0, 'Other_drugs': 1}] = [0.3, 0.7]
bn.cpt(R)[{'Insulin': 1, 'Other_drugs': 1}] = [0.9, 0.1]
bn.cpt(R)

(pyAgrum.Potential<double>@0x600001527280) 
             ||  Remission        |
Insuli|Other_||0        |1        |
------|------||---------|---------|
0     |0     || 0.1000  | 0.9000  |
1     |0     || 0.6000  | 0.4000  |
0     |1     || 0.3000  | 0.7000  |
1     |1     || 0.9000  | 0.1000  |

### Show the graphe

In [41]:
bn

(pyAgrum.BayesNet<double>@0x7fae00564c00) BN{nodes: 4, arcs: 4, domainSize: 16, dim: 18}

### Save BN net work

In [42]:
print(gum.availableBNExts())
gum.saveBN(bn,"Diabetes_Remission.bif")
with open("Diabetes_Remission.bif","r") as out:
    print(out.read())

bif|dsl|net|bifxml|o3prm|uai
network "Diabetes Remission" {
// written by aGrUM 0.22.5
}

variable Glycemia {
   type discrete[2] {0, 1};
}

variable Insulin {
   type discrete[2] {0, 1};
}

variable Other_drugs {
   type discrete[2] {0, 1};
}

variable Remission {
   type discrete[2] {0, 1};
}

probability (Glycemia) {
   default 0.5 0.5;
}
probability (Insulin | Glycemia) {
   (0) 0.9 0.1;
   (1) 0.1 0.9;
}
probability (Other_drugs | Glycemia) {
   (0) 0.75 0.25;
   (1) 0.3 0.7;
}
probability (Remission | Insulin, Other_drugs) {
   (0, 0) 0.1 0.9;
   (1, 0) 0.6 0.4;
   (0, 1) 0.3 0.7;
   (1, 1) 0.9 0.1;
}




### Inference without evidence

In [15]:
ie=gum.LazyPropagation(bn)
ie.makeInference()
print (ie.posterior("Remission"))


  Remission        |
0        |1        |
---------|---------|
 0.4903  | 0.5097  |



### Inference with evidence

Q: What is the probability to get the remission if the
glycemia level is less than 6.5 and no drugs are taken?

In [23]:
ie.setEvidence({'Glycemia':1, 'Other_drugs': 1})
ie.makeInference()
ie.posterior("Remission")

(pyAgrum.Potential<double>@0x600001534da0) 
  Remission        |
0        |1        |
---------|---------|
 0.8400  | 0.1600  |

A: In this case, the probability to get the remission is 0.84.

Q: What is the probability to get the remission if the glycemia level is bigger than 6.5 and
insulin is prescribed?

In [24]:
ie.setEvidence({'Glycemia':0, 'Insulin': 0})
ie.makeInference()
ie.posterior("Remission")

(pyAgrum.Potential<double>@0x600001523320) 
  Remission        |
0        |1        |
---------|---------|
 0.1500  | 0.8500  |

A: In this case, the probability to get remission is 0.15.

## Construct Bayesian networks from real data

In [79]:
import pandas as pd
from pyAgrum.lib.image import export

data_env = pd.read_csv('data/SPLEX_env.txt' , sep=' ')
data_host = pd.read_csv('data/SPLEX_host.txt' , sep=' ')
data_joint = pd.concat([data_env,data_host],axis=1)

In [94]:
def to_csv(data, file):
    l=[]
    for col in data.columns.values:
        bins = np.linspace(min(data[col]), max(data[col]), 5)
        l.append(pd.DataFrame(np.digitize(data[col], bins),columns=[col]))
        
    discr_data = pd.concat(l, join='outer', axis=1)
    discr_data.to_csv(file,index=False)

### Env

In [100]:
to_csv(data_env, "data/discr_data_env.csv")

In [101]:
learner=gum.BNLearner("data/discr_data_env.csv")
learner.useLocalSearchWithTabuList()
bn_l = learner.learnBN()
export(bn_l,"data/env_Local_search.pdf")

learner=gum.BNLearner("data/discr_data_env.csv")
learner.useGreedyHillClimbing()
bn_G = learner.learnBN()
export(bn_G,"data/env_Hill_Climbing.pdf")

### Host

In [102]:
to_csv(data_host, "data/discr_data_host.csv")

In [103]:
learner=gum.BNLearner("data/discr_data_host.csv")
learner.useLocalSearchWithTabuList()
bn_l = learner.learnBN()
export(bn_l,"data/host_Local_search.pdf")

learner=gum.BNLearner("data/discr_data_host.csv")
learner.useGreedyHillClimbing()
bn_G = learner.learnBN()
export(bn_G,"data/host_Hill_Climbing.pdf")

### Joint

In [104]:
to_csv(data_host, "data/discr_data_joint.csv")

In [105]:
learner=gum.BNLearner("data/discr_data_joint.csv")
learner.useLocalSearchWithTabuList()
bn_l = learner.learnBN()
export(bn_l,"data/joint_Local_search.pdf")

learner=gum.BNLearner("data/discr_data_joint.csv")
learner.useGreedyHillClimbing()
bn_G = learner.learnBN()
export(bn_G,"data/joint_Hill_Climbing.pdf")